## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [1]:
!pip install pytextrank
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import spacy
import pytextrank

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir -p ~/.aws
!cp "/content/drive/My Drive/aws_config" ~/.aws/config

In [0]:
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

def get_summarized_review(review, nlp):
  doc = nlp(review)
  summary = ''
  for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=5):
    summary += str(sent)
  return summary


In [0]:
import boto3 
nlp_client = boto3.client('comprehend', region_name='us-east-1')

In [0]:
def get_sentiment_with_scores_from_aws(review):
  try:
    summarized_review = get_summarized_review(review, nlp)
    result = nlp_client.detect_sentiment(Text=summarized_review, LanguageCode='en')
    sentiment = result['Sentiment']
    scores_dict = result['SentimentScore']
  except nlp_client.exceptions.TextSizeLimitExceededException:
    print('The batch was too long')
    sentiment = None
    scores_dict = None

  return sentiment, scores_dict  

In [10]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [0]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

In [12]:
len(reviews_neg)

1000

### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [0]:
def score_review(review):
    # TBD: call the service and return 'pos' or 'neg'
    sentiment, scores_dict = get_sentiment_with_scores_from_aws(review)
    print(f'sentiment: {sentiment}, scores_dict: {scores_dict}')
    if sentiment.lower() == 'positive':      
      return 'pos'
    elif sentiment.lower() == 'negative':
      return 'neg'
    else:
      if int(scores_dict['Positive']) >= int(scores_dict['Negative']):
        return 'pos'
      return 'neg'

In [21]:
reviews_pos_tmp = list(reviews_pos[:10])
reviews_neg_tmp = list(reviews_neg[:10])

# len(reviews_pos_tmp)
# len(results_neg_tmp)

results_pos_tmp = []
for review in reviews_pos_tmp:
    result = score_review(review)
    results_pos_tmp.append(result)

print('positive reviews done.')

results_neg_tmp = []
for review in reviews_neg_tmp:
    result = score_review(review)
    results_neg_tmp.append(result)

print('negative reviews done.')

print(results_pos_tmp)
print(results_neg_tmp)


sentiment: MIXED, scores_dict: {'Positive': 0.07248460501432419, 'Negative': 0.04214577004313469, 'Neutral': 0.018267836421728134, 'Mixed': 0.867101788520813}
sentiment: NEGATIVE, scores_dict: {'Positive': 0.006018439307808876, 'Negative': 0.9669187664985657, 'Neutral': 0.026386043056845665, 'Mixed': 0.000676744501106441}
sentiment: POSITIVE, scores_dict: {'Positive': 0.5415773987770081, 'Negative': 0.034649815410375595, 'Neutral': 0.36752620339393616, 'Mixed': 0.05624666064977646}
sentiment: NEUTRAL, scores_dict: {'Positive': 0.35716721415519714, 'Negative': 0.0451183021068573, 'Neutral': 0.526157021522522, 'Mixed': 0.0715574100613594}
sentiment: NEUTRAL, scores_dict: {'Positive': 0.17016345262527466, 'Negative': 0.10923541337251663, 'Neutral': 0.7109896540641785, 'Mixed': 0.009611480869352818}
sentiment: NEUTRAL, scores_dict: {'Positive': 0.12526719272136688, 'Negative': 0.009778941981494427, 'Neutral': 0.8648654818534851, 'Mixed': 8.83562388480641e-05}
sentiment: POSITIVE, scores_di

### Score each review

Now, we can use the function you defined to score each of the reviews

In [22]:
results_pos = []
for review in reviews_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
for review in reviews_neg:
    result = score_review(review)
    results_neg.append(result)

sentiment: MIXED, scores_dict: {'Positive': 0.07248460501432419, 'Negative': 0.04214577004313469, 'Neutral': 0.018267836421728134, 'Mixed': 0.867101788520813}
sentiment: NEGATIVE, scores_dict: {'Positive': 0.006018439307808876, 'Negative': 0.9669187664985657, 'Neutral': 0.026386043056845665, 'Mixed': 0.000676744501106441}
sentiment: POSITIVE, scores_dict: {'Positive': 0.5415773987770081, 'Negative': 0.034649815410375595, 'Neutral': 0.36752620339393616, 'Mixed': 0.05624666064977646}
sentiment: NEUTRAL, scores_dict: {'Positive': 0.35716721415519714, 'Negative': 0.0451183021068573, 'Neutral': 0.526157021522522, 'Mixed': 0.0715574100613594}
sentiment: NEUTRAL, scores_dict: {'Positive': 0.17016345262527466, 'Negative': 0.10923541337251663, 'Neutral': 0.7109896540641785, 'Mixed': 0.009611480869352818}
sentiment: NEUTRAL, scores_dict: {'Positive': 0.12526719272136688, 'Negative': 0.009778941981494427, 'Neutral': 0.8648654818534851, 'Mixed': 8.83562388480641e-05}
sentiment: POSITIVE, scores_di

In [24]:
len(results_neg)

1000

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [25]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 88.3% correct
Negative reviews: 37.0% correct
Overall accuracy: 62.64999999999999% correct
